### Importando dados de bancos SQL

Utilizando o `pymssql`

In [6]:
import pandas as pd
import numpy as np
import pymssql
import mysql.connector

- Selecionando as colunas que queremos do SQL Server e usando SQL **aliases** para melhorar o nome das colunas.
- Criando uma conexão com o SQL Server passando as credenciais para a função `pymssql`.
- Criando um dataframe Pandas passando uma query e a conexão para o objeto `read_sql`

In [7]:
query = "SELECT studentid, school, sex, age, famsize, \
    medu AS mothereducation, fedu AS fathereducation, \
        traveltime, studytime, failures, famrel, freetime, \
            goout, g1 AS gradeperiod1, g2 AS gradeperiod2, \
                g3 AS gradeperiod3 FROM studentmath"

In [8]:
server = "pdcc.c9sqqzd5fulv.us-west-2.rds.amazonaws.com"
user = "pdccuser"
password = "pdccpass"
database = "pdcctest"
conn = pymssql.connect(server=server, user=user, password=password, database=database)

In [10]:
studentmath = pd.read_sql(query,conn)
conn.close()

Checando os dados e as primeiras linhas.

In [12]:
studentmath.dtypes

studentid          object
school             object
sex                object
age                 int64
famsize            object
mothereducation     int64
fathereducation     int64
traveltime          int64
studytime           int64
failures            int64
famrel              int64
freetime            int64
goout               int64
gradeperiod1        int64
gradeperiod2        int64
gradeperiod3        int64
dtype: object

In [13]:
studentmath.head()

,studentid,school,sex,age,famsize,mothereducation,fathereducation,traveltime,studytime,failures,famrel,freetime,goout,gradeperiod1,gradeperiod2,gradeperiod3
0,001,GP,F,18,GT3,4,4,2,2,0,4,3,4,5,6,6
1,002,GP,F,17,GT3,1,1,1,2,0,5,3,3,5,5,6
2,003,GP,F,15,LE3,1,1,1,2,3,4,3,2,7,8,10
3,004,GP,F,15,GT3,4,2,1,3,0,3,2,2,15,14,15
4,005,GP,F,16,GT3,3,3,1,2,0,4,3,2,6,10,10


Utilizando o `mysql connector`

In [14]:
host = "pdccmysql.c9sqqzd5fulv.us-west-2.rds.amazonaws.com"
user = "pdccuser"
password = "pdccpass"
database = "pdccschema"

connmysql = mysql.connector.connect(host=host, database=database, user=user, password=password)

studentmath = pd.read_sql(query,connmysql)
connmysql.close()

/home/robson/.local/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Re-arranjando as colunas, com index e checando campos sem valores.

In [15]:
newcolorder = ['studentid', 'gradeperiod1', 'gradeperiod2', 'gradeperiod3', 'school', 'sex',\
    'age', 'famsize', 'mothereducation', 'fathereducation', 'traveltime', 'studytime', 'freetime',\
        'failures', 'famrel', 'goout']

studentmath = studentmath[newcolorder]

In [16]:
studentmath.studentid.count()

395

In [17]:
studentmath.studentid.nunique()

395

In [18]:
studentmath.set_index('studentid', inplace=True)

In [19]:
studentmath.count()

gradeperiod1       395
gradeperiod2       395
gradeperiod3       395
school             395
sex                395
age                395
famsize            395
mothereducation    395
fathereducation    395
traveltime         395
studytime          395
freetime           395
failures           395
famrel             395
goout              395
dtype: int64

Substituindo alguns valores do dataframe por valores mais informativos.

In [20]:
setvalues = {
    "famrel":{1:"1:very bad", 2:"2:bad", 3:"3:neutral", 4:"4:good", 5:"5:excellent"},
    "freetime":{1:"1:very low", 2:"2:low", 3:"3:neutral", 4:"4:hight", 5:"5:very hight"},
    "goout":{1:"1:very low", 2:"2:low", 3:"3:neutral", 4:"4:hight", 5:"5:very hight"},
    "mothereducation":{0:np.nan, 1:"1:k4", 2:"2:5-9", 3:"3:secondary ed", 4:"4:higher ed"},
    "fathereducation":{0:np.nan, 1:"1:k4", 2:"2:5-9", 3:"3:secondary ed", 4:"4:higher ed"}}

In [21]:
studentmath.replace(setvalues, inplace=True)

In [24]:
setvalueskeys = [k for k in setvalues]

- Mudando o tipo para colunas com dados modificados para `category`.
- Checando qualquer modificação no uso de memória

In [25]:
studentmath[setvalueskeys].memory_usage(index=False)

famrel             3160
freetime           3160
goout              3160
mothereducation    3160
fathereducation    3160
dtype: int64

In [26]:
for col in studentmath[setvalueskeys].columns:
    studentmath[col] = studentmath[col].astype('category')

In [27]:
studentmath[setvalueskeys].memory_usage(index=False)

famrel             607
freetime           607
goout              607
mothereducation    599
fathereducation    599
dtype: int64